In [1]:
from ibapi.wrapper import EWrapper
from ibapi.client import EClient
from ibapi.contract import Contract as IBcontract
from ibapi.order import Order
from ibapi.execution import ExecutionFilter
from threading import Thread
from importlib import reload as rel
import numpy as np, pandas as pd, datetime as dt, time
import queue, tools, collections
import HistMktData as hmd, OrderPlacement as op

#### Simulated inputs (from extension)

In [2]:
underlying = 'EURUSD'
msg = 'n=entryL1 d=long t=l p=0 q=1'

In [15]:
app = op.TestApp("127.0.0.1", 7497, 2) # Connection
ibcontract, order1 = tools.parse_message(app, underlying, msg)
app.disconnect() # Disconnection

Getting full contract details from the server... 
12087792,EUR,CASH,,0.0,,,IDEALPRO,,USD,EUR.USD,EUR.USD,False,,combo:
0,0,0: LMT BUY 1@1.140000 DAY


#### Historical Market Data

In [5]:
app = hmd.TestApp("127.0.0.1", 7497, 1) # Connection
ibcontract, minTick = tools.make_contract(app, underlying)
hist_mkt_data = tools.get_hist_data(app, ibcontract, durationStr='2 D', barSizeSetting='1 hour')
app.disconnect() # Disconnection

Getting full contract details from the server... 
Getting historical data from the server... could take up to 8 seconds to complete 
IB error id -1 errorcode 2106 string HMDS data farm connection is OK:cashhmds


#### Order Placement

In [18]:
app = op.TestApp("127.0.0.1", 7497, 1)

## resolve the contract
resolved_ibcontract = app.resolve_ib_contract(ibcontract)

order1 = Order()
order1.action = "BUY"
order1.orderType = "MKT"
order1.totalQuantity = 25000
order1.transmit = True

orderid1 = app.place_new_IB_order(resolved_ibcontract, order1, orderid=None)

print("Placed market order, orderid is %d" % orderid1)

while app.any_open_orders():
    ## Warning: this will pause forever if fill doesn't come back
    time.sleep(1)

## Have a look at the fill
print("Recent fills")
filldetails = app.recent_fills_and_commissions()
print(filldetails)

## when I call again should be empty as we've cleared the memory of recent fills
print("Recent fills (should be blank)")
morefilldetails = app.recent_fills_and_commissions()
print(morefilldetails)

## but this won't be
print("Executions today")
execdetails = app.get_executions_and_commissions()
print(execdetails)

## And here is a limit order, unlikely ever to be filled
## Note sell limit price of 2.00

order2 = Order()
order2.action = "SELL"
order2.orderType = "LMT"
order2.totalQuantity = 30000
order2.lmtPrice = 2.00
order2.tif = 'DAY'
order2.transmit = True


orderid2 = app.place_new_IB_order(resolved_ibcontract, order2, orderid=None)
print("Placed limit order, orderid is %d" % orderid2)

## Short wait ...
time.sleep(5)

print("Open orders (should be one)")
open_orders = app.get_open_orders()
print(open_orders)

## put in another limit order
order3 = Order()
order3.action = "BUY"
order3.orderType = "LMT"
order3.totalQuantity = 35000
order3.lmtPrice = 1.00
order3.tif = 'DAY'
order3.transmit = True

orderid3 = app.place_new_IB_order(resolved_ibcontract, order3, orderid=None)
print("Placed limit order, orderid is %d" % orderid3)

print("Open orders (should be two)")
open_orders = app.get_open_orders()
print(open_orders.keys())

time.sleep(5)

## modify the order

print("Modifying order %d" % orderid3)

order3.lmtPrice = 1.05

print("Limit price was %f will become %f" % (open_orders[orderid3].order.lmtPrice, order3.lmtPrice ))

app.place_new_IB_order(resolved_ibcontract, order3, orderid=orderid3)
time.sleep(5)
open_orders = app.get_open_orders()
print("New limit price %f " % open_orders[orderid3].order.lmtPrice)

## Cancel a single limit order, leaving one active limit order (order3)
print("Cancel order %d " % orderid2)
app.cancel_order(orderid2)
open_orders = app.get_open_orders()

print("Open orders (should just be %d)" % orderid2)
print(open_orders.keys())

print("Cancelling all orders")
app.cancel_all_orders()

print("Any open orders? - should be False")
print(app.any_open_orders())

app.disconnect()

IB error id -1 errorcode 504 string Not connected
Exceeded maximum wait for wrapper to confirm finished - seems to be normal behaviour
Failed to get additional contract details: returning unresolved contract
Getting orderid from IB


KeyboardInterrupt: 